In [2]:
# ================================================
# IMPORTS
# ================================================
import os
import re
import json
import numpy as np
from dotenv import load_dotenv
from mistralai import Mistral
import faiss
import time
import random

# ================================================
# 0. Charger clé API
# ================================================
load_dotenv()
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)


# ================================================
# 1. Fonctions utilitaires
# ================================================
def show_tree(root):
    """Génère l'arborescence en texte brut."""
    lines = []
    for dirpath, dirnames, filenames in os.walk(root):
        level = dirpath.replace(root, "").count(os.sep)
        indent = " " * 4 * level
        lines.append(f"{indent}{os.path.basename(dirpath)}/")
        subindent = " " * 4 * (level + 1)
        for f in filenames:
            lines.append(f"{subindent}{f}")
    return "\n".join(lines)


def read_code(file_path):
    """Lit .py ou .ipynb et renvoie le code."""
    ext = os.path.splitext(file_path)[1].lower()

    if ext == ".ipynb":
        with open(file_path, "r", encoding="utf-8") as f:
            nb = json.load(f)
            content = []
            for cell in nb.get("cells", []):
                if cell.get("cell_type") == "code":
                    content.append("".join(cell.get("source", [])))
            return "\n".join(content)

    else:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()


def split_functions(code):
    """Découpe le code en fonctions/chunks."""
    pattern = r"(def [\w_]+\s*\(.*?\):(?:\n(?:\s+.+))*)"
    chunks = re.findall(pattern, code, re.DOTALL)
    return chunks if chunks else [code]


def extract_imports(code):
    """Extrait toutes les bibliothèques importées."""
    imports = set()
    for line in code.splitlines():
        line = line.strip()
        if line.startswith("import "):
            imports.add(line.replace("import ", "").split()[0])
        elif line.startswith("from "):
            imports.add(line.split()[1])
    return imports


# ================================================
# 2. Embeddings + gestion des erreurs
# ================================================
def create_embeddings(chunks, batch_size=16, max_retries=5):
    embeddings = []

    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        for attempt in range(max_retries):
            try:
                resp = client.embeddings.create(
                    model="codestral-embed",
                    inputs=batch
                )
                for emb in resp.data:
                    embeddings.append(np.array(emb.embedding, dtype=np.float32))
                break

            except Exception as e:
                if "429" in str(e):
                    wait = 2 ** attempt + random.random()
                    print(f"Erreur 429… Retry dans {wait:.2f}s")
                    time.sleep(wait)
                else:
                    raise e

    return embeddings


# ================================================
# 3. Pipe RAG : indexation + génération README
# ================================================
def generate_readme_RAG(folder_path, output_file):

    # ---- 3.1 Lire fichiers code ----
    code_ext = {".py", ".js", ".ts", ".cpp", ".c", ".java", ".ipynb"}
    all_files = []
    all_codes = []

    for dirpath, _, filenames in os.walk(folder_path):
        for f in filenames:
            if os.path.splitext(f)[1].lower() in code_ext:
                fp = os.path.join(dirpath, f)
                code = read_code(fp)
                if code.strip():
                    all_files.append(fp)
                    all_codes.append(code)

    if not all_codes:
        raise ValueError("Aucun fichier code trouvé dans ce dossier !")

    # ---- 3.2 Chunking ----
    chunks = []
    chunk_paths = []

    for file_path, code in zip(all_files, all_codes):
        ch = split_functions(code)
        chunks.extend(ch)
        chunk_paths.extend([file_path]*len(ch))

    print(f"Total chunks : {len(chunks)}")

    # ---- 3.3 Embeddings ----
    embeddings = create_embeddings(chunks, batch_size=16)
    dim = len(embeddings[0])

    # ---- 3.4 Indexation FAISS ----
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))

    # ---- 3.5 Arborescence ----
    tree = show_tree(folder_path)

    # ---- 3.6 Construire le prompt RAG final ----
    prompt = f"""
Tu es un expert en analyse de code. 
Voici une liste de chunks extraits du projet.

Ton rôle :
- Reconstituer le sens du projet
- Faire un README.md professionnel
- Inclure : 
    # Titre du projet
    # Présentation générale
    # Arborescence du dossier
    # Bibliothèques nécessaires
    # Fonctionnement global
    # Résumé fichier par fichier
    # Comment lancer le projet

Voici l'arborescence du dossier :
-----------------
{tree}
-----------------

Voici les chunks utiles (non ordonnés) :
-----------------
{chunks[:100]}  # On n'en envoie qu'un échantillon
-----------------

Génère maintenant un README Markdown complet et propre.
"""

    response = client.chat.complete(
        model="codestral-2508",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=2000
    )

    readme_text = response.choices[0].message.content

    # ---- 3.7 Sauvegarde ----
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(readme_text)

    print("\n README généré :", output_file)


# ================================================
# 4. DEMANDE UTILISATEUR
# ================================================
folder = input("Dossier à analyser : ").strip()
output = os.path.join(folder, "README.md")

if os.path.exists(output):
    os.remove(output)

generate_readme_RAG(folder, output)


Total chunks : 35
Erreur 429… Retry dans 1.00s
Erreur 429… Retry dans 1.53s
Erreur 429… Retry dans 2.04s
Erreur 429… Retry dans 4.51s

✔ README généré : C:\Users\33611\Documents\WePlant_25.5\README.md


Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\code3.py ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 1\code1.py ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 1\code5.ipynb ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 2\code2.py ...
Analyse du fichier D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\DOSSIER 2\code4.ipynb ...
 README.md généré ici : D:\33611\Documents\MASTER\M2\PROJET_README\creating-a-README-for-a-codebase\TESTS\TEST2\README.md
